In [3]:
import pandas as pd
import requests
import bs4
import json
import csv

In [ ]:
def get_data_gov_map():
    '''
    Get all the real estate deals in Tel-Aviv from GovMap - from 1998 until now.
    50000000-50006000 include all the range of the street codes in Tel Aviv (note: part of them are empty).
    This function gets JSON data and returns a DataFrame with all the data.
    '''
    url =  "https://ags.govmap.gov.il/Identify/GetDealsByStreetCodes"
    xmin, ymin, xmax, ymax = 185110.8086, 669110.696799999, 185730.7648, 669715.339299999
    street_codes = list(range(50000000, 50006000))
    data_list = []
    
    for i, street_code in enumerate(street_codes):
        payload = {
            "StreetCodes": [street_code],
            "Extent": {
                "xmin": xmin,
                "ymin": ymin,
                "xmax": xmax,
                "ymax": ymax
            }
        }

        try:
            data = requests.post(url, json=payload, timeout=10).json()
            data_list.append(data["data"])
            print(f'index: {i}, street code: {street_code}, shape: {len(data["data"])}')
        except:
            print(f"Oops! StreetCodes:{street_code} isn't working")

    df = pd.concat([pd.DataFrame(data) for data in data_list], ignore_index=True)
    return df
    

In [1]:
def get_nandlan_data(url, payload, df):
    '''
    This function gets all real estate deals in Tel-Aviv from nadlan.gov.il - from 1998 until now.
    '''
    page_no = 3901
    while page_no < 5200:
        print(page_no)
        try:
            response = requests.post(url, json=payload)
            json_data = response.json()
            new_df = pd.DataFrame(json_data['AllResults'])
            if new_df.shape[0] != 0:
                df = pd.concat([df, new_df], ignore_index=True)
        except:
            print(f"Error: {page_no}")
            time.sleep(30)
        
        page_no += 1
        payload['PageNo'] = page_no
        
    return df


In [2]:
url =  "https://www.nadlan.gov.il/Nadlan.REST/Main/GetAssestAndDeals"

payload = {
    "MoreAssestsType": 0,
    "FillterRoomNum": 0,
    "GridDisplayType": 0,
    "ResultLable": "תל אביב -יפו",
    "ResultType": 1,
    "ObjectID": "5000",
    "ObjectIDType": "text",
    "ObjectKey": "UNIQ_ID",
    "DescLayerID": "SETL_MID_POINT",
    "Alert": None,
    "X": 180428.31832654,
    "Y": 665726.5550939,
    "Gush": "",
    "Parcel": "",
    "showLotParcel": False,
    "showLotAddress": False,
    "OriginalSearchString": "תל אביב  יפו",
    "MutipuleResults": False,
    "ResultsOptions": None,
    "CurrentLavel": 2,
    "Navs": [],
    "QueryMapParams": {
        "QueryToRun": None,
        "QueryObjectID": "5000",
        "QueryObjectType": "number",
        "QueryObjectKey": "SETL_CODE",
        "QueryDescLayerID": "KSHTANN_SETL_AREA",
        "SpacialWhereClause": None
    },
    "isHistorical": False,
    "PageNo": 3900,
    "OrderByFilled": "DEALDATETIME",
    "OrderByDescending": True,
    "Distance": 0
}
# 4000 - 2009
# 4200 - 2008
# 4500 - 2007
# 5000 - 2005 
# 5100 - 2004 

(151025, 22)

In [ ]:
response = requests.post(url, json=payload)
json_data = response.json()
df = pd.DataFrame(json_data['AllResults'])
df = get_nandlan_data(url , payload ,df )

In [ ]:
# Save the Data into csv files

def split_and_save_df(df):
    '''
    This funtion use to split the dataframe og GOVMAPS beacuse of the size of the csv file
    '''
    middle_row = len(df) // 2
    
    # Split the DataFrame into two new DataFrames
    df1 = df.iloc[:middle_row]
    df2 = df.iloc[middle_row:]
    
    # Save the two DataFrames as CSV files
    df1.to_csv("Data/Real_Estate_TLV_GOVMAPS_1.csv", index=False)
    df2.to_csv("Data/Real_Estate_TLV_GOVMAPS_2.csv", index=False)
    
    


gov_map_df = get_data_gov_map()
split_and_save_df(gov_map_df)

nadlan_df = get_nandlan_data(url,payload,df)
nadlan_df.to_csv('Data/Real_Estate_TLV_NADLAN.csv')